# Examples for a basic query against OEDI for ResStock

## Import modules

In [1]:
from buildstock_query import BuildStockQuery
import pandas as pd

## Reload changes
If you are a developer and are making changes to the code, signal to reload changes each time a block is executed

In [32]:
# auto reload
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Initialize BuildStock Query Object
Connect to the database, setup the tables, set the source schema, and set project workgroup

In [33]:
my_run = BuildStockQuery(
    'rescore', # Change to your project working group
    'buildstock_sdr',
    (
        "resstock_2024_amy2018_release_2_metadata_state_vu", # Baseline metadata view
        "resstock_2024_amy2018_release_2_by_state_vu", # Timeseries view
        "resstock_2024_amy2018_release_2_metadata_state_vu" # Upgrade metadata view (same as Baseline for OEDI)
    ),
    db_schema="resstock_oedi",
    skip_reports=True
)

INFO:buildstock_query.query_core:Loading ('resstock_2024_amy2018_release_2_metadata_state_vu', 'resstock_2024_amy2018_release_2_by_state_vu', 'resstock_2024_amy2018_release_2_metadata_state_vu') ...
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


## Simple annual query

In [ ]:
baseline_agg = my_run.agg.aggregate_annual(
    enduses=['electricity.total.energy_consumption'],
    group_by=['state', 'geometry_building_type_recs'],
)
baseline_agg

,in.state,geometry_building_type_recs,sample_count,units_count,electricity.total.energy_consumption
0,AL,Mobile Home,1233,3.110879e+05,3.841222e+09
1,AL,Multi-Family with 2 - 4 Units,450,1.135357e+05,1.098778e+09
2,AL,Multi-Family with 5+ Units,1071,2.702151e+05,2.040843e+09
3,AL,Single-Family Attached,143,3.607913e+04,4.530630e+08
4,AL,Single-Family Detached,6223,1.570073e+06,2.567576e+10
...,...,...,...,...,...
239,WY,Mobile Home,150,3.784525e+04,2.784647e+08
240,WY,Multi-Family with 2 - 4 Units,74,1.867032e+04,1.422061e+08
241,WY,Multi-Family with 5+ Units,103,2.598707e+04,1.829168e+08
242,WY,Single-Family Attached,53,1.337199e+04,1.306436e+08


## Simple timeseries query

In [35]:
ts_agg = my_run.agg.aggregate_timeseries(
    enduses=['out.electricity.total.energy_consumption'],
    restrict=[('state', ['CO'])],
    upgrade_id=0,
    timestamp_grouping_func='month',
    group_by=['geometry_building_type_recs', 'state'],
    get_query_only=False
)
print(ts_agg)

INFO:buildstock_query.aggregate_query:Restricting query to Upgrade 0.


      geometry_building_type_recs state  timestamp  sample_count  \
0                     Mobile Home    CO 2018-01-01           391   
1                     Mobile Home    CO 2018-02-01           391   
2                     Mobile Home    CO 2018-03-01           391   
3                     Mobile Home    CO 2018-04-01           391   
4                     Mobile Home    CO 2018-05-01           391   
5                     Mobile Home    CO 2018-06-01           391   
6                     Mobile Home    CO 2018-07-01           391   
7                     Mobile Home    CO 2018-08-01           391   
8                     Mobile Home    CO 2018-09-01           391   
9                     Mobile Home    CO 2018-10-01           391   
10                    Mobile Home    CO 2018-11-01           391   
11                    Mobile Home    CO 2018-12-01           391   
12  Multi-Family with 2 - 4 Units    CO 2018-01-01           469   
13  Multi-Family with 2 - 4 Units    CO 2018-02-

## Utility unit count query

In [ ]:
units = my_run.utility.aggregate_unit_counts_by_eiaid(
    eiaid_list=['4110', '14328'], # ComEd and PG&E,
    get_query_only=False,
    group_by=['state']
)
print(units)
units.to_csv("utility_units.csv")

INFO:buildstock_query.utility_query:Aggregating unit counts by eiaid


   eiaid in.state  sample_count   units_count
0  14328       CA         28730  1.828840e+09
1   4110       IL         15880  1.010859e+09


## Utility annual query

In [ ]:
enduses = [
    'out.site_energy.net.energy_consumption',
    'out.site_energy.total.energy_consumption'
]

# By default just does all the eiaids
ts_utility_agg = my_run.utility.aggregate_annual_by_eiaid(
    enduses=enduses,
    group_by=['state'],
    get_query_only=False
)
print(ts_utility_agg)
ts_utility_agg.to_csv("utility_annual_agg.csv")

      eiaid state  sample_count   units_count  \
0     10000    KS          1774  1.129260e+08   
1     10000    MO          2453  1.561485e+08   
2     10005    KS          1801  1.146447e+08   
3     10009    TX          3206  2.040815e+08   
4     10012    OK           319  2.030630e+07   
...     ...   ...           ...           ...   
1901   9964    KY           658  4.188572e+07   
1902    998    KS           108  6.874861e+06   
1903   9991    MN           385  2.450761e+07   
1904   9996    KS           278  1.769640e+07   
1905   9999    IN          1976  1.257845e+08   

      site_energy.net.energy_consumption  site_energy.total.energy_consumption  
0                           4.152121e+12                          4.152489e+12  
1                           5.237433e+12                          5.242919e+12  
2                           3.853915e+12                          3.853915e+12  
3                           3.782175e+12                          3.807804e+12  
4     

## Utility timeseries query

In [40]:
enduses = [
    'out.site_energy.net.energy_consumption',
    'out.site_energy.total.energy_consumption'
]
group_by=['state', 'timestamp'] # If no timestamp, then annual values will be returned

ts_utility_agg = my_run.utility.aggregate_ts_by_eiaid(
    eiaid_list=['4110', '14328'], # ComEd and PG&E
    enduses=enduses,
    group_by=group_by,
    get_query_only=False
)
print(ts_utility_agg)
ts_utility_agg.to_csv("utility_ts_agg.csv")

INFO:buildstock_query.utility_query:Will submit request for ['4110', '14328']


INFO:buildstock_query.utility_query:Submitting query for ['4110', '14328']
INFO:buildstock_query.aggregate_query:Restricting query to Upgrade 0.
INFO:buildstock_query.query_core:{'submitted': 0, 'running': 0, 'pending': 1, 'completed': 0, 'failed': 0}
INFO:buildstock_query.query_core:Submitted queries[0] (53663a66-a9aa-4386-9687-70e88aee3557)
INFO:buildstock_query.query_core:{'submitted': 1, 'running': 0, 'pending': 1, 'completed': 0, 'failed': 0}
INFO:buildstock_query.query_core:{'submitted': 1, 'running': 1, 'pending': 0, 'completed': 0, 'failed': 0}
INFO:buildstock_query.query_core:{'submitted': 1, 'running': 1, 'pending': 0, 'completed': 0, 'failed': 0}
INFO:buildstock_query.query_core:{'submitted': 1, 'running': 1, 'pending': 0, 'completed': 0, 'failed': 0}
INFO:buildstock_query.query_core:{'submitted': 1, 'running': 1, 'pending': 0, 'completed': 0, 'failed': 0}
INFO:buildstock_query.query_core:{'submitted': 1, 'running': 1, 'pending': 0, 'completed': 0, 'failed': 0}
INFO:buildsto

       eiaid state           timestamp  sample_count   units_count  \
0      14328    CA 2018-01-01 00:15:00         28730  3.417140e+06   
1      14328    CA 2018-01-01 00:30:00         28730  3.417140e+06   
2      14328    CA 2018-01-01 00:45:00         28730  3.417140e+06   
3      14328    CA 2018-01-01 01:00:00         28730  3.417140e+06   
4      14328    CA 2018-01-01 01:15:00         28730  3.417140e+06   
...      ...   ...                 ...           ...           ...   
70075   4110    IL 2018-12-31 23:00:00         15880  2.563967e+06   
70076   4110    IL 2018-12-31 23:15:00         15880  2.563967e+06   
70077   4110    IL 2018-12-31 23:30:00         15880  2.563967e+06   
70078   4110    IL 2018-12-31 23:45:00         15880  2.563967e+06   
70079   4110    IL 2019-01-01 00:00:00         15880  2.563967e+06   

       site_energy.net.energy_consumption  \
0                            2.730670e+06   
1                            2.796730e+06   
2                       